# Evaluating the Effectiveness of Few-Shot Learning in Traffic Sign Recognition Tasks

<h2>ABSTRACT</h2>
<p style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">Nowadays, self-driving veichles is one of the main application of Artificial Intelligence (AI) which has been most widely used in daily-life scenarios. These veichles are based on many AI models among which there are traffic sign recognition systems. Different proposals have been provided for these systems, most of which rely on the usage of big datasets such as GTSRB (dataset of German traffic signs). Collecting a big amount of data is one of the main problems of Machine Learning models, which need lots of evidences in order to being able to generalize well. The effort in collecting more data than available can be one of the main aspects to consider during the development of these systems. Furthermore, using a big dataset expose to possible Adversarial attacks, from poisoning to evasion ones, specially in case of out-source training and testing. For these reasons, in this work we investigate the application of state-of-art Few-shot learning models on the traffic sign recognition domain. The aim of this work is to define the effectiveness of these approaches, in contrast to non-few-shot ones, to open the possibilities of build more robust and effortless solutions in this field.</p>

# NOTE
più peso alla motivazione dell'attacco rispetto alla difficoltà nella raccolta di dati. Quindi focalizzarsi sul problema che attacchi di poisoning necessitano di aggiungere esempi poisonati che sono facilmente scopribili in casi di dataset piccoli (come nel few-shot) mediante un controllo umano dei sample. Attenzione: verificare, in caso questo controllo non si possa fare, se l'attacco è avvantaggiato con un dataset più piccolo rispetto ad uno grande. Se cosi fosse, sarebbe un problema.

in the related work there are the works we are trying to improve

specificare come ho creato il dataset ridotto

testare i modelli anche sul dataset ridotto

cercare come vengono valutati i modelli di few shot. Se si usa bootstrap o altre tecniche

usare più modelli few shot, non solo CLIP e LDC

-  N-way-k-shot (it means N classes and K instances per classes)

-  Approfondire discorso support set (set non utilizzato nè durante training nè durante test ma solo durante fase di predizione)


# RICERCHE ESISTENTI
- <b>FUSED-Net: Enhancing Few-Shot Traffic Sign Detection with Unfrozen Parameters, Pseudo-Support Sets, Embedding Normalization, and Domain Adaptation</b>: https://arxiv.org/html/2409.14852v1 SOLO SU DATASET DEL BANGLADESH

- <b>Cross-domain Few-shot In-context Learning for Enhancing Traffic Sign Recognition</b>: questo fa un compare tra modelli few shot sul dataset GTSRB

- <b>Few-shot traffic sign recognition with clustering inductive bias and random neural network</b>: questo propone un nuovo modello per few-shot su traffic sign recognition

- <b>Meta-YOLO: Meta-Learning for Few-Shot Traffic Sign Detection via Decoupling Dependencies</b>: questo fa traffic sign detection e non recognition (usa YOLO) quindi a noi non ci interessa

- <b>Research on a Traffic Sign Recognition Method under Small Sample Conditions</b>: nuovo modello proposto

-  <b>Self-supervised few-shot learning for real-time traffic sign classification</b>: altro modello introdotto

<h2>No few-shot models used for the comparison</h2>
<div style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">
<ul>
<li>An effective automatic traffic sign classification and recognition deep convolutional networks (2022) (Quello di Mattia)
<a href="https://doi.org/10.1007/s11042-022-12531-w">Link Paper</a>
</li>
<li>Multi-column deep neural network for traffic sign classification (2012) (Modello MCDNN)
<a href="https://doi.org/10.1016/j.neunet.2012.02.023">Link Paper</a>
<br><a href="https://github.com/sixftninja/german-traffic-sign-detection/tree/master">Link GitHub</a>
</li>
<li>A Low-cost and Ultra-lightweight Binary Neural Network for Traffic Signal Recognition (2025) (Modello BNN) (No GitHub)
<a href="http://dx.doi.org/10.48550/arXiv.2501.07808">Link Paper</a>
</li>
<li>Visual Trasformer (No Paper, solo codice)
<a href="https://huggingface.co/kelvinandreas/vit-traffic-sign-GTSRB">Link HuggingFace</a>
</li>
</ul>
</div>

<h2>Few-shot models used for the comparison</h2>
<div style="text-align: justify; font-family: Arial; font-size: 15px; line-height: 1.6; font-weight: normal;margin-right: 20px;">
<ul>
<li>(Quello nostro di Computer Vision) Clip encoder (<a href="https://doi.org/10.48550/arXiv.2103.00020">Link Paper</a>) + LCD to calculate the noise (<a href="https://doi.org/10.48550/arXiv.2504.12104">Link Paper</a>)
</li>
</ul>
</div>




# EVALUATION METRICS

Possible evaluation metrics used in the papers above are:
Ma precision e recall non sono per binary classification? Lo dovresti fare per ogni classe considerando tutte le altre classi come istanze negative?
- Average precision (AP): area under the precision-recall curve
- Precision
- Recall
- A sto punto anche F1 Score? Male non fa
- Accuracy rate (consigliato al posto di error rate. più utilizzato in letteratura)
-

# EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS
- <b>Bootstrap</b>
- <b>Fine-Grained Image Classification</b>: si valutano e testano gruppi di classi alla volta in modo tale da identificare i gruppi più problematici (nel nostro caso il gruppo dei segnali stradali relativi ai limiti di velocità potrebbero essere piu difficili da classificare rispetto a quelli di pericolo). "Fine-Grained Image Classification is a task in computer vision where the goal is to classify images into subcategories within a larger category. For example, classifying different species of birds or different types of flowers. This task is considered to be fine-grained because it requires the model to distinguish between subtle differences in visual appearance and patterns, making it more challenging than regular image classification tasks."(https://paperswithcode.com/task/fine-grained-image-classification)
-<b>Generalization error</b>: How much does accuracy drop when we reduce K (e.g., from K=5 to K=1)?*
- <b>Paper on few-shot evaluation techniques</b> https://arxiv.org/pdf/2307.02732

# EVALUATION TECHNIQUES TO COMPARE THE FEW SHOT VS MANY-SHOT LEARNING
- <b>Paired T-test</b>: Usiamo J dataset differenti per il traffic sign recgnition task (uno è quello tedesco, dobbiamo trovarne altri) e testiamo per ciascuno di essi il modello few-shot ed uno no-few shot. Per ottenere l'error rate per ciascun dataset possiamore usare l'holdout o cross-validation (meglio ques'ultimo perchè citato nel paper come migliore metodo di valutazione per modelli few-shot).
1) Usiamo l'holdout assicurandoci di avere stesso identico test set per entrambi i modelli ma training set diverso (per il modello few-shot ovviamente abbiamo bisogno di meno sample rispetto a quello no few-shot).
2) In alternativa all'holdout, possiamo usare una k-fold cross validation in questo modo: il numero e la suddivisione dei fold sono gli stessi per entrambi i modelli, tuttavia, ad ogni iterazione della cross validation, usiamo lo stesso test set per entrambi e, per il modello few-shot, anzichè usare tutte i rimanenti fold per il training set ne prendiamo solo una parte. Dato che il numero di sample per classe nel modello few-shot segue la potenza del 2(Y=2,4,8,16 tipicamente), allora dobbiamo fare in modo che ogni fold abbia Y*N(dove N è il numero delle label e Y il numero dei sample per label richiesto dal modello few-shot) con l'accortezza di averne almeno x=Y/2 che siano strutturati in modo tale da avere SOLTANTO Y istanze per ciascuna delle N label. Problema: cosi abbiamo pochi sample nel test set dato che i fold sono piccoli per venire incontro alle esigenze del modello few-shot (J*N sample in ciascun fold)...Soluzione: anzichè prendere un fold alla volta per il test set ne prendiamo tipo 3 (ovviamente ciò vale per entrambi i modelli). Quindi per il no few-shot prendiamo X fold per il test set e utilizziamo i restanti per il training set, mentre per il few-shot prendiamo sempre X fold per il test set e utilizziamo soltanto quelli che ci servono il training. 


- <b>Intervalli di confidenza</b> A parte dal t-test, per ogni confronto tra due algoritmi ci calcoliamo per entrambe il corrispondente intervallo di confidenza.
- <b>Focus su few-shot</b> A parte dal t-test, testiamo anche il modello few-shot sui diversi gruppi di classi (fine-grained few shot) (no train, solo test) per identificare su quali gruppi il modello few-shot si comporta peggio.





# Nuova sezione

<h2>Code used to create the random dataset:</h2>
import os
import random
import shutil
from PIL import Image

def copy_random_images(p1, p2, num_images_per_folder=5):
    """
    Copies random images from subfolders in p1 to p2, recreating the subfolder structure.

    Args:
        p1 (str): Source path containing subfolders with images.
        p2 (str): Destination path where the subfolder structure and images will be recreated.
        num_images_per_folder (int): Number of random images to copy from each subfolder.
    """
    if not os.path.exists(p1):
        raise ValueError(f"Source path '{p1}' does not exist.")
    
    if not os.path.exists(p2):
        os.makedirs(p2)

    # Iterate through subfolders in p1
    for subfolder in os.listdir(p1):
        subfolder_path = os.path.join(p1, subfolder)
        if os.path.isdir(subfolder_path):
            # Create the same subfolder in p2
            dest_subfolder_path = os.path.join(p2, subfolder)
            os.makedirs(dest_subfolder_path, exist_ok=True)

            # Get all image files in the current subfolder
            image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
            
            # Select random images
            random_images = random.sample(image_files, min(num_images_per_folder, len(image_files)))

            # Copy selected images to the destination subfolder
            for image in random_images:
                src_image_path = os.path.join(subfolder_path, image)
                dest_image_path = os.path.join(dest_subfolder_path, image)

                # Open, resize, and save the image
                with Image.open(src_image_path) as img:
                    img = img.convert("RGB")  # Ensure the image is in RGB format
                    img = img.resize((32, 32))  # Resize to 32x32
                    img.save(dest_image_path)

            print(f"Copied {len(random_images)} images from '{subfolder_path}' to '{dest_subfolder_path}'.")

p1 = "ortiginal_images_folder"
p2 = "destination_images_folder"
num_images_per_folder = 30

arr=["/train","/val","/test"]

for a in arr:
  copy_random_images(p1, p2, num_images_per_folder+a)

# Setup environment

<h2> LOAD THE DATASETS </h2>

In [12]:
from datasets import load_dataset, DatasetDict, Dataset as LoadDataset, Image,concatenate_datasets
import pandas as pd
import os
import kagglehub
import shutil
import matplotlib.pyplot as plt
import io
from PIL import Image as ImagePil
import gc
from collections.abc import Iterator 
from typing import Optional
from tqdm.notebook import tqdm





class Dataset:
    def __init__(self, dataset_path, dataset_name, column_name_image, column_name_label):
        """Initialize a Dataset object
        
        Args:
            dataset_path (str): path of the dataset
            dataset_name (str): name of the dataset
            column_name_image(str) the name of the column in which there are the images
            column_name_label(str): the name of the column in which there are the labels
        """

        # Inizializza gli attributi privati
        self._dataset_path = None
        self._dataset_name = None
        self._column_name_image = None
        self._column_name_label = None
        self._pandas_dataset = None
        
        # Usa i setter per validazione
        self.dataset_path = dataset_path
        self.dataset_name = dataset_name
        self.column_name_image = column_name_image
        self.column_name_label = column_name_label
        self.pandas_dataset = None


   
    # ---- Dataset Path ----
    @property
    def dataset_path(self) -> Optional[str]:
        return self._dataset_path
    
    @dataset_path.setter
    def dataset_path(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("dataset_path deve essere una stringa")
        self._dataset_path = value


    # ---- Pandas dataset ----
    @property
    def pandas_dataset(self):
        return self._pandas_dataset 
    
    @pandas_dataset.setter
    def pandas_dataset(self, value) -> None:
        self._pandas_dataset = value  

    # ---- Dataset Name ----
    @property
    def dataset_name(self) -> Optional[str]:
        return self._dataset_name
    
    @dataset_name.setter
    def dataset_name(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("dataset_name deve essere una stringa")
        if not value.strip():
            raise ValueError("dataset_name non può essere vuoto")
        self._dataset_name = value

    # ---- Image Column ----
    @property
    def column_name_image(self) -> Optional[str]:
        return self._column_name_image
    
    @column_name_image.setter
    def column_name_image(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("column_name_image deve essere una stringa")
        self._column_name_image = value

    # ---- Label Column ----
    @property
    def column_name_label(self) -> Optional[str]:
        return self._column_name_label
    
    @column_name_label.setter
    def column_name_label(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("column_name_label deve essere una stringa")
        self._column_name_label = value
    

class LocalDataset(Dataset):
    pass

class HuggingFaceDataset(Dataset):
    pass

class KaggleDataset(Dataset):
    def __init__(self, dataset_path, dataset_name, column_name_image, column_name_label, local_subpath):
        """Initialize a KaggleDataset object
        
        Args:
            dataset_path (str): path of the dataset
            dataset_name (str): name of the dataset
            column_name_image(str) the name of the column in which there are the images
            column_name_label(str): the name of the column in which there are the labels
            local_subpath (str): subpath to use to find the actual directory containing the images
        """
        super().__init__(dataset_path, dataset_name, column_name_image, column_name_label)
        self._local_subpath = None  # Inizializza prima
        self.local_subpath = local_subpath  # Poi usa il setter

    @property
    def local_subpath(self) -> Optional[str]:
        return self._local_subpath
    
    @local_subpath.setter
    def local_subpath(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("local_subpath deve essere una stringa")
        self._local_subpath = value


class DatasetCollection:

    kaggle_dataset_path = "./dataset/Kaggle"
    hugging_face_dataset_local_path = "./dataset/HuggingFace"
    templates = {
            "belgian_dataset":[
    # Class 0: Speed limit 30
    "A white circular sign with a red border, with the number '30' in the center.",
    # Class 1: Speed limit 50
    "A white circular sign with a red border, with the number '50' in the center.",
    # Class 2: Speed limit 70
    "A white circular sign with a red border, with the number '70' in the center.",
    # Class 3: Speed limit 80
    "A white circular sign with a red border, with the number '80' in the center.",
    # Class 4: No overtaking
    "A white circular sign with a red border, showing a red car and a black car side-by-side.",
    # Class 5: No overtaking for trucks
    "A white circular sign with a red border, showing a red truck and a black car side-by-side.",
    # Class 6: Priority road
    "A yellow, diamond-shaped sign with a white border.",
    # Class 7: End of priority road
    "A yellow, diamond-shaped sign with a white border, crossed out by a single black diagonal stripe.",
    # Class 8: Give way
    "An inverted triangular sign with a red border and a white background.",
    # Class 9: Stop
    "A red, octagonal sign with the word 'STOP' in white letters.",
    # Class 10: No entry for all vehicles
    "A plain white circular sign with a red border.",
    # Class 11: End of speed limit 30 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '30'.",
    # Class 12: End of speed limit 50 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '50'.",
    # Class 13: End of speed limit 70 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '70'.",
    # Class 14: End of speed limit 80 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '80'.",
    # Class 15: End of no overtaking
    "A white circular sign with multiple black diagonal stripes, showing faded silhouettes of two cars.",
    # Class 16: End of no overtaking for trucks
    "A white circular sign with multiple black diagonal stripes, showing faded silhouettes of a truck and a car.",
    # Class 17: Speed limit 90
    "A white circular sign with a red border, with the number '90' in the center.",
    # Class 18: Road works
    "A triangular sign with a red border, containing a black silhouette of a person with a shovel.",
    # Class 19: Speed limit 60
    "A white circular sign with a red border, with the number '60' in the center.",
    # Class 20: Dangerous curve to the right
    "A triangular sign with a red border, showing a black arrow that curves to the right.",
    # Class 21: Dangerous curve to the left
    "A triangular sign with a red border, showing a black arrow that curves to the left.",
    # Class 22: General caution / Danger
    "A triangular sign with a red border, containing a black exclamation mark.",
    # Class 23: No entry
    "A red circular sign with a solid white horizontal bar across its center.",
    # Class 24: End of speed limit 60 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '60'.",
    # Class 25: End of speed limit 90 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '90'.",
    # Class 26: Speed limit 120
    "A white circular sign with a red border, with the number '120' in the center.",
    # Class 27: End of speed limit 120 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '120'.",
    # Class 28: Speed limit 40
    "A white circular sign with a red border, with the number '40' in the center.",
    # Class 29: End of speed limit 40 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '40'.",
    # Class 30: No left turn
    "A white circular sign with a red border, showing a black arrow turning left, crossed out by a red diagonal line.",
    # Class 31: No right turn
    "A white circular sign with a red border, showing a black arrow turning right, crossed out by a red diagonal line.",
    # Class 32: Bumpy road
    "A triangular sign with a red border, showing a single raised bump in a horizontal line.",
    # Class 33: Roundabout
    "A triangular sign with a red border, showing three black arrows forming a circle.",
    # Class 34: Slippery road
    "A triangular sign with a red border, showing a car silhouette with curving skid marks.",
    # Class 35: Road narrows
    "A triangular sign with a red border, showing two vertical lines curving inwards.",
    # Class 36: Pedestrian crossing
    "A triangular sign with a red border, featuring a silhouette of a person on a crosswalk.",
    # Class 37: Children crossing
    "A triangular sign with a red border, showing black silhouettes of two people, one larger and one smaller.",
    # Class 38: End of all prohibitions
    "A plain white circular sign with a single black diagonal stripe across it.",
    # Class 39: Ahead only
    "A blue circular sign with a white arrow pointing straight upwards.",
    # Class 40: Turn right ahead
    "A blue circular sign with a white arrow pointing to the right.",
    # Class 41: Turn left ahead
    "A blue circular sign with a white arrow pointing to the left.",
    # Class 42: No parking
    "A blue circular sign with a red border and a single red diagonal line.",
    # Class 43: Two-way traffic
    "A triangular sign with a red border, showing two black arrows pointing in opposite vertical directions.",
    # Class 44: Traffic signals ahead
    "A triangular sign with a red border, containing a symbol of a traffic light.",
    # Class 45: Go straight or right
    "A blue circular sign displaying two white arrows, one pointing up and one pointing to the right.",
    # Class 46: Bicycles crossing
    "A triangular sign with a red border, containing a black silhouette of a bicycle.",
    # Class 47: Keep right
    "A blue circular sign with a white arrow pointing diagonally down and to the right.",
    # Class 48: Speed limit 20
    "A white circular sign with a red border, with the number '20' in the center.",
    # Class 49: Speed limit 100
    "A white circular sign with a red border, with the number '100' in the center.",
    # Class 50: End of speed limit 20 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '20'.",
    # Class 51: End of speed limit 100 zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number '100'.",
    # Class 52: No U-turn
    "A white circular sign with a red border, showing a black U-shaped arrow, crossed out by a red diagonal line.",
    # Class 53: No stopping or parking
    "A blue circular sign with a red border and a red 'X' cross.",
    # Class 54: No entry for cyclists
    "A white circular sign with a red border, containing a black silhouette of a bicycle.",
    # Class 55: No entry for pedestrians
    "A white circular sign with a red border, containing a black silhouette of a person walking.",
    # Class 56: Mandatory roundabout
    "A blue circular sign with three white arrows forming a counter-clockwise circle.",
    # Class 57: End of built-up area
    "A white rectangular sign with a black silhouette of a city skyline, crossed out by a red diagonal line.",
    # Class 58: Beginning of built-up area
    "A white rectangular sign with a black silhouette of a city skyline.",
    # Class 59: Go straight or left
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the left.",
    # Class 60: Keep left
    "A blue circular sign with a white arrow pointing diagonally down and to the left.",
    # Class 61: No entry for trucks
    "A white circular sign with a red border, containing a black silhouette of a truck."
            ],
            "russian_dataset":[
    # Class 0: 1.1_railroad_crossing_with_barrier
    "A triangular sign with a red border, containing a black silhouette of a fence or gate.",
    # Class 1: 1.2_railroad_crossing_without_barrier
    "A triangular sign with a red border, containing a black silhouette of a steam train.",
    # Class 2: 1.5_intersection_with_a_tram_line
    "A triangular sign with a red border, containing a black silhouette of a tram.",
    # Class 3: 1.6_crossing_of_equivalent_roads
    "A triangular sign with a red border, showing a black cross symbol.",
    # Class 4: 1.8_traffic_lights
    "A triangular sign with a red border, containing a symbol of a traffic light.",
    # Class 5: 1.16_uneven_road
    "A triangular sign with a red border, showing two bumps in a horizontal line.",
    # Class 6: 1.17_road_bump
    "A triangular sign with a red border, showing a single raised bump in a horizontal line.",
    # Class 7: 1.19_slippery_road
    "A triangular sign with a red border, showing a car silhouette with curving skid marks.",
    # Class 8: 1.20_road_narrows
    "A triangular sign with a red border, showing two vertical lines curving inwards.",
    # Class 9: 1.22_pedestrian_crossing
    "A triangular sign with a red border, featuring a silhouette of a person on a crosswalk.",
    # Class 10: 1.23_children
    "A triangular sign with a red border, showing silhouettes of two running children.",
    # Class 11: 1.25_road_works
    "A yellow triangular sign with a red border, containing a black silhouette of a person with a shovel.",
    # Class 12: 1.30_wild_animals
    "A triangular sign with a red border, showing the silhouette of a leaping deer.",
    # Class 13: 1.34.1_direction_of_turn
    "A rectangular sign with a red background, showing thick white chevron arrows.",
    # Class 14: 1.34.2_direction_of_turn
    "A rectangular sign with a red background, showing thick white chevron arrows.",
    # Class 15: 1.34.3_direction_of_turn
    "A rectangular sign with a red background, showing thick white chevron arrows.",
    # Class 16: 2.1_main_road
    "A yellow, diamond-shaped sign with a white border.",
    # Class 17: 2.2_end_of_main_road
    "A yellow, diamond-shaped sign with a white border, crossed out by four thin black diagonal lines.",
    # Class 18: 2.4_yield
    "An inverted triangular sign with a red border and a white background.",
    # Class 19: 2.5_stop
    "A red, octagonal sign with the word 'СТОП' in white Cyrillic letters.",
    # Class 20: 2.7_advantage_for_oncoming_traffic
    "A white circular sign with a red border, showing a black arrow and a red arrow pointing in opposite directions.",
    # Class 21: 3.1_no_entry
    "A red circular sign with a solid white horizontal bar across its center.",
    # Class 22: 3.2_no_vehicles
    "A plain white circular sign with a red border.",
    # Class 23: 3.4_no_trucks
    "A white circular sign with a red border, containing a black silhouette of a truck.",
    # Class 24: 3.11_weight_limit
    "A white circular sign with a red border, showing a number followed by 'т' (the Cyrillic letter for 't').",
    # Class 25: 3.12_axle_load_limit
    "A white circular sign with a red border, showing a number above a small icon of a vehicle axle.",
    # Class 26: 3.13_height_limit
    "A white circular sign with a red border, showing a number between two vertically pointing arrows.",
    # Class 27: 3.14_width_limit
    "A white circular sign with a red border, showing a number between two horizontally pointing arrows.",
    # Class 28: 3.16_length_limit
    "A white circular sign with a red border, showing a number next to a silhouette of a long vehicle.",
    # Class 29: 3.18.1_no_right_turn
    "A white circular sign with a red border, showing a black arrow turning right, crossed out by a red diagonal line.",
    # Class 30: 3.18.2_no_left_turn
    "A white circular sign with a red border, showing a black arrow turning left, crossed out by a red diagonal line.",
    # Class 31: 3.19_no_u_turn
    "A white circular sign with a red border, showing a black U-shaped arrow, crossed out by a red diagonal line.",
    # Class 32: 3.20_no_overtaking
    "A white circular sign with a red border, showing a red car and a black car side-by-side.",
    # Class 33: 3.21_end_of_no_overtaking_zone
    "A white circular sign with multiple black diagonal stripes, showing faded silhouettes of two cars.",
    # Class 34: 3.24_speed_limit
    "A white circular sign with a red border, with a number in the center.",
    # Class 35: 3.25_end_of_speed_limit_zone
    "A white circular sign with multiple black diagonal stripes, showing a faded number.",
    # Class 36: 3.27_no_stopping
    "A blue circular sign with a red border and a red 'X' cross.",
    # Class 37: 3.28_no_parking
    "A blue circular sign with a red border and a single red diagonal line.",
    # Class 38: 3.31_end_of_all_restrictions
    "A white circular sign with multiple thin black diagonal stripes.",
    # Class 39: 4.1.1_go_straight
    "A blue circular sign with a white arrow pointing straight upwards.",
    # Class 40: 4.1.2_go_right
    "A blue circular sign with a white arrow pointing to the right.",
    # Class 41: 4.1.3_go_left
    "A blue circular sign with a white arrow pointing to the left.",
    # Class 42: 4.1.4_go_straight_or_right
    "A blue circular sign displaying two white arrows, one pointing up and one pointing to the right.",
    # Class 43: 4.1.5_go_straight_or_left
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the left.",
    # Class 44: 4.2.1_keep_right
    "A blue circular sign with a white arrow curving around an obstacle on the left.",
    # Class 45: 4.2.2_keep_left
    "A blue circular sign with a white arrow curving around an obstacle on the right.",
    # Class 46: 4.3_roundabout
    "A blue circular sign with three white arrows forming a counter-clockwise circle.",
    # Class 47: 5.5_one_way_road
    "A blue square sign with a thick, long white arrow pointing upwards.",
    # Class 48: 5.6_end_of_one_way_road
    "A blue square sign with a thick, long white arrow pointing upwards, crossed out by a red diagonal line.",
    # Class 49: 5.15.1_lane_directions
    "A blue rectangular sign with white arrows indicating permissible directions of travel from different lanes.",
    # Class 50: 5.15.2_lane_directions
    "A blue rectangular sign with white arrows indicating permissible directions of travel from different lanes.",
    # Class 51: 5.15.7_lane_directions
    "A blue square sign with white arrows showing lane directions.",
    # Class 52: 5.19.1_pedestrian_crossing
    "A blue square sign containing a white triangle with a silhouette of a person on a crosswalk.",
    # Class 53: 5.19.2_pedestrian_crossing
    "A blue square sign containing a white triangle with a silhouette of a person on a crosswalk.",
    # Class 54: 5.21_residential_area
    "A blue rectangular sign showing white silhouettes of a house, a car, and a person with a child.",
    # Class 55: 6.8.1_dead_end
    "A blue square sign with a white 'T' shape that has a thick red horizontal bar.",
    # Class 56: 8.22.1_obstacle
    "A rectangular white sign with diagonal black and white stripes pointing downwards.",
    # Class 57: 8.23_photo_and_video_recording
    "A rectangular white sign with a black silhouette of a camera."
            ],
            "german_dataset":[
    # Class 0: Speed limit (20km/h)
    "A white circular sign with a red border, with the number '20' in the center.",
    # Class 1: Speed limit (30km/h)
    "A white circular sign with a red border, with the number '30' in the center.",
    # Class 2: Speed limit (50km/h)
    "A white circular sign with a red border, with the number '50' in the center.",
    # Class 3: Speed limit (60km/h)
    "A white circular sign with a red border, with the number '60' in the center.",
    # Class 4: Speed limit (70km/h)
    "A white circular sign with a red border, with the number '70' in the center.",
    # Class 5: Speed limit (80km/h)
    "A white circular sign with a red border, with the number '80' in the center.",
    # Class 6: End of speed limit (80km/h)
    "A white circular sign with multiple black diagonal stripes, showing a faded number '80'.",
    # Class 7: Speed limit (100km/h)
    "A white circular sign with a red border, with the number '100' in the center.",
    # Class 8: Speed limit (120km/h)
    "A white circular sign with a red border, with the number '120' in the center.",
    # Class 9: No passing
    "A white circular sign with a red border, showing a red car and a black car side-by-side.",
    # Class 10: No passing for vehicles over 3.5 tons
    "A white circular sign with a red border, showing a red truck and a black car side-by-side.",
    # Class 11: Right-of-way at the next intersection
    "A triangular sign with a red border, showing a thick black vertical arrow intersected by a thinner horizontal line.",
    # Class 12: Priority road
    "A yellow, diamond-shaped sign with a white border.",
    # Class 13: Yield
    "An inverted triangular sign with a red border and a white background.",
    # Class 14: Stop
    "A red, octagonal sign with the word 'STOP' in white letters.",
    # Class 15: No vehicles
    "A plain white circular sign with a red border.",
    # Class 16: Vehicles over 3.5 tons prohibited
    "A white circular sign with a red border, containing a black silhouette of a truck.",
    # Class 17: No entry
    "A red circular sign with a solid white horizontal bar across its center.",
    # Class 18: General caution
    "A triangular sign with a red border, containing a black exclamation mark.",
    # Class 19: Dangerous curve to the left
    "A triangular sign with a red border, showing a black arrow that curves to the left.",
    # Class 20: Dangerous curve to the right
    "A triangular sign with a red border, showing a black arrow that curves to the right.",
    # Class 21: Double curve
    "A triangular sign with a red border, displaying a black S-shaped line.",
    # Class 22: Bumpy road
    "A triangular sign with a red border, showing a single raised bump in a horizontal line.",
    # Class 23: Slippery road
    "A triangular sign with a red border, showing a car silhouette with curving skid marks.",
    # Class 24: Road narrows on the right
    "A triangular sign with a red border, showing a straight vertical line on the left and a curving line on the right.",
    # Class 25: Road work
    "A triangular sign with a red border, containing a black silhouette of a person with a shovel.",
    # Class 26: Traffic signals
    "A triangular sign with a red border, containing a symbol of a traffic light.",
    # Class 27: Pedestrians
    "A triangular sign with a red border, featuring the silhouette of a person walking across a series of horizontal lines.",
    # Class 28: Children crossing
    "A triangular sign with a red border, showing black silhouettes of two people, one larger and one smaller.",
    # Class 29: Bicycles crossing
    "A triangular sign with a red border, containing a black silhouette of a bicycle.",
    # Class 30: Beware of ice/snow
    "A triangular sign with a red border, containing a black snowflake symbol.",
    # Class 31: Wild animals crossing
    "A triangular sign with a red border, showing the silhouette of a leaping deer.",
    # Class 32: End of all speed and passing limits
    "A white circular sign with a single black diagonal stripe across it.",
    # Class 33: Turn right ahead
    "A blue circular sign with a white arrow pointing to the right.",
    # Class 34: Turn left ahead
    "A blue circular sign with a white arrow pointing to the left.",
    # Class 35: Ahead only
    "A blue circular sign with a white arrow pointing straight upwards.",
    # Class 36: Go straight or right
    "A blue circular sign displaying two white arrows, one pointing up and one pointing to the right.",
    # Class 37: Go straight or left
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the left.",
    # Class 38: Keep right
    "A blue circular sign with a white arrow pointing diagonally down and to the right.",
    # Class 39: Keep left
    "A blue circular sign with a white arrow pointing diagonally down and to the left.",
    # Class 40: Roundabout mandatory
    "A blue circular sign with three white arrows forming a counter-clockwise circle.",
    # Class 41: End of no passing
    "A white circular sign with multiple black diagonal stripes, showing faded silhouettes of two cars.",
    # Class 42: End of no passing by vehicles over 3.5 tons
    "A white circular sign with multiple black diagonal stripes, showing faded silhouettes of a truck and a car."
            ],
            "chinese_dataset":[
    # Class 0: prohibitory_speed_limit_20
    "A white circular sign with a red border, with the number '20' in the center.",
    # Class 1: prohibitory_speed_limit_30
    "A white circular sign with a red border, with the number '30' in the center.",
    # Class 2: prohibitory_speed_limit_40
    "A white circular sign with a red border, with the number '40' in the center.",
    # Class 3: prohibitory_speed_limit_50
    "A white circular sign with a red border, with the number '50' in the center.",
    # Class 4: prohibitory_speed_limit_60
    "A white circular sign with a red border, with the number '60' in the center.",
    # Class 5: prohibitory_speed_limit_70
    "A white circular sign with a red border, with the number '70' in the center.",
    # Class 6: prohibitory_speed_limit_80
    "A white circular sign with a red border, with the number '80' in the center.",
    # Class 7: prohibitory_no_entry_for_motor_vehicles
    "A white circular sign with a red border, containing a black silhouette of a car.",
    # Class 8: prohibitory_no_entry_for_hgvs
    "A white circular sign with a red border, containing a black silhouette of a truck.",
    # Class 9: prohibitory_no_entry
    "A red circular sign with a solid white horizontal bar across its center.",
    # Class 10: prohibitory_no_honking
    "A white circular sign with a red border, containing a black silhouette of a horn.",
    # Class 11: prohibitory_no_left_turn
    "A white circular sign with a red border, showing a black arrow turning left, crossed out by a red diagonal line.",
    # Class 12: prohibitory_no_right_turn
    "A white circular sign with a red border, showing a black arrow turning right, crossed out by a red diagonal line.",
    # Class 13: prohibitory_no_u_turn
    "A white circular sign with a red border, showing a black U-shaped arrow, crossed out by a red diagonal line.",
    # Class 14: prohibitory_no_overtaking
    "A white circular sign with a red border, showing a red car and a black car side-by-side.",
    # Class 15: prohibitory_no_parking
    "A blue circular sign with a red border and a single red diagonal line.",
    # Class 16: warning_crossroads
    "A triangular sign with a red border, showing a black cross symbol.",
    # Class 17: warning_narrow_road_both_sides
    "A triangular sign with a red border, showing two vertical lines curving inwards.",
    # Class 18: warning_narrow_road_right
    "A triangular sign with a red border, showing a straight vertical line on the left and a curving line on the right.",
    # Class 19: warning_rail_crossing_with_barriers
    "A triangular sign with a red border, containing a black silhouette of a fence or gate.",
    # Class 20: warning_rail_crossing_without_barriers
    "A triangular sign with a red border, containing a black silhouette of a steam train.",
    # Class 21: warning_attention
    "A triangular sign with a red border, containing a black exclamation mark.",
    # Class 22: warning_pedestrian
    "A triangular sign with a red border, containing a black silhouette of a person walking.",
    # Class 23: warning_school
    "A triangular sign with a red border, showing black silhouettes of two people, one larger and one smaller.",
    # Class 24: warning_bicycles
    "A triangular sign with a red border, containing a black silhouette of a bicycle.",
    # Class s25: warning_road_works
    "A triangular sign with a red border, containing a black silhouette of a person with a shovel.",
    # Class 26: warning_traffic_lights
    "A triangular sign with a red border, containing a symbol of a traffic light with red, yellow, and green circles.",
    # Class 27: warning_sharp_curve_to_the_left
    "A triangular sign with a red border, showing a black arrow that sharply curves to the left.",
    # Class 28: warning_sharp_curve_to_the_right
    "A triangular sign with a red border, showing a black arrow that sharply curves to the right.",
    # Class 29: warning_winding_road
    "A triangular sign with a red border, displaying a black S-shaped line.",
    # Class 30: warning_steep_ascent
    "A triangular sign with a red border, showing a car silhouette on an upward slope.",
    # Class 31: warning_steep_descent
    "A triangular sign with a red border, showing a car silhouette on a downward slope.",
    # Class 32: warning_uneven_road
    "A triangular sign with a red border, showing two bumps in a horizontal line.",
    # Class 33: warning_hump_bridge
    "A triangular sign with a red border, showing a single raised bump in a horizontal line.",
    # Class 34: warning_village
    "A triangular sign with a red border, showing a silhouette of several buildings.",
    # Class 35: warning_falling_rocks
    "A triangular sign with a red border, showing rocks falling from a cliff on one side.",
    # Class 36: warning_side_wind
    "A triangular sign with a red border, containing a black silhouette of a windsock.",
    # Class 37: warning_unprotected_quayside
    "A triangular sign with a red border, showing a car silhouette falling off a ledge into water.",
    # Class 38: warning_tunnel
    "A triangular sign with a red border, showing a black silhouette of a tunnel entrance.",
    # Class 39: warning_domestic_animals
    "A triangular sign with a red border, containing a black silhouette of a cow.",
    # Class 40: warning_slow
    "A triangular sign with a red border, containing Chinese characters.",
    # Class 41: mandatory_turn_left
    "A blue circular sign with a white arrow pointing to the left.",
    # Class 42: mandatory_turn_right
    "A blue circular sign with a white arrow pointing to the right.",
    # Class 43: mandatory_go_straight
    "A blue circular sign with a white arrow pointing straight upwards.",
    # Class 44: mandatory_go_straight_or_turn_left
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the left.",
    # Class 45: mandatory_go_straight_or_turn_right
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the right.",
    # Class 46: mandatory_keep_left
    "A blue circular sign with a white arrow pointing diagonally down and to the left.",
    # Class 47: mandatory_keep_right
    "A blue circular sign with a white arrow pointing diagonally down and to the right.",
    # Class 48: mandatory_roundabout
    "A blue circular sign with three white arrows forming a counter-clockwise circle.",
    # Class 49: mandatory_end_of_no_overtaking
    "A white circular sign with a black diagonal stripe, showing faded silhouettes of two cars.",
    # Class 50: mandatory_horn
    "A blue circular sign with a white silhouette of a horn.",
    # Class 51: mandatory_bicycle_lane
    "A blue circular sign with a white silhouette of a bicycle.",
    # Class 52: mandatory_speed_limit_40
    "A blue circular sign with the number '40' in white.",
    # Class 53: mandatory_speed_limit_50
    "A blue circular sign with the number '50' in white.",
    # Class 54: mandatory_speed_limit_60
    "A blue circular sign with the number '60' in white.",
    # Class 55: other_stop
    "A red, octagonal sign with white characters.",
    # Class 56: other_parking
    "A square blue sign with a large white letter 'P' and other symbols or characters.",
    # Class 57: other_give_way
    "An inverted triangular sign with a red border, a white background, and black characters."
            ],
            "indian_dataset":[
    # ClassId 0
    "An inverted triangular sign with a red border and a white background.",
    # ClassId 1
    "A red circular sign with a solid white horizontal bar across its center.",
    # ClassId 2
    "A rectangular blue sign with a large white arrow pointing in one direction.",
    # ClassId 3
    "A rectangular blue sign with a large white arrow pointing in one direction.",
    # ClassId 4
    "A plain white circular sign with a red border.",
    # ClassId 5
    "A white circular sign with a red border, containing a black silhouette of a bicycle.",
    # ClassId 6
    "A white circular sign with a red border, containing a black silhouette of a truck.",
    # ClassId 7
    "A white circular sign with a red border, containing a black silhouette of a person walking.",
    # ClassId 8
    "A white circular sign with a red border, containing a black silhouette of an animal-drawn cart.",
    # ClassId 9
    "A white circular sign with a red border, containing a black silhouette of a person pushing a cart.",
    # ClassId 10
    "A white circular sign with a red border, containing a black silhouette of a car.",
    # ClassId 11
    "A white circular sign with a red border, showing a number between two vertically pointing arrows.",
    # ClassId 12
    "A white circular sign with a red border, showing a number followed by 't' for tonnes.",
    # ClassId 13
    "A white circular sign with a red border, showing a number above a small icon of a vehicle axle.",
    # ClassId 14
    "A white circular sign with a red border, showing a number next to a silhouette of a long vehicle.",
    # ClassId 15
    "A white circular sign with a red border, showing a black arrow turning left, crossed out by a red diagonal line.",
    # ClassId 16
    "A white circular sign with a red border, showing a black arrow turning right, crossed out by a red diagonal line.",
    # ClassId 17
    "A white circular sign with a red border, showing a red car and a black car side-by-side.",
    # ClassId 18
    "A white circular sign with a red border, with the number '90' in the center.",
    # ClassId 19
    "A white circular sign with a red border, with the number '110' in the center.",
    # ClassId 20
    "A white circular sign with a red border, containing a black silhouette of a horn.",
    # ClassId 21
    "A blue circular sign with a red border and a single red diagonal line.",
    # ClassId 22
    "A blue circular sign with a red border and a red 'X' cross.",
    # ClassId 23
    "A blue circular sign with a white arrow pointing to the left.",
    # ClassId 24
    "A blue circular sign with a white arrow pointing to the right.",
    # ClassId 25
    "A triangular sign with a red border, showing a car silhouette on a downward slope with a percentage.",
    # ClassId 26
    "A triangular sign with a red border, showing a car silhouette on an upward slope with a percentage.",
    # ClassId 27
    "A triangular sign with a red border, showing two vertical lines curving inwards.",
    # ClassId 28
    "A triangular sign with a red border, showing two vertical lines that narrow in the middle.",
    # ClassId 29
    "A triangular sign with a red border, showing a car silhouette falling off a ledge into water.",
    # ClassId 30
    "A triangular sign with a red border, showing a single raised bump in a horizontal line.",
    # ClassId 31
    "A triangular sign with a red border, showing a single dip in a horizontal line.",
    # ClassId 32
    "A triangular sign with a red border, showing a car silhouette with small specks flying from a tire.",
    # ClassId 33
    "A triangular sign with a red border, showing rocks falling from a cliff on one side.",
    # ClassId 34
    "A triangular sign with a red border, containing a black silhouette of a cow.",
    # ClassId 35
    "A triangular sign with a red border, showing a black cross symbol.",
    # ClassId 36
    "A triangular sign with a red border, showing a main vertical line with a smaller line joining from one side.",
    # ClassId 37
    "A triangular sign with a red border, showing a main vertical line with a smaller line joining from one side.",
    # ClassId 38
    "A triangular sign with a red border, showing a main vertical line with a smaller diagonal line joining from one side.",
    # ClassId 39
    "A triangular sign with a red border, showing a main vertical line with a smaller diagonal line joining from one side.",
    # ClassId 40
    "A triangular sign with a red border, showing a vertical line ending at a horizontal line, forming a 'T' shape.",
    # ClassId 41
    "A triangular sign with a red border, showing a black 'Y' shape.",
    # ClassId 42
    "A triangular sign with a red border, showing a vertical line with two offset horizontal lines joining from opposite sides.",
    # ClassId 43
    "A triangular sign with a red border, showing a vertical line with two offset horizontal lines joining from opposite sides.",
    # ClassId 44
    "A triangular sign with a red border, showing three arrows forming a circle.",
    # ClassId 45
    "A triangular sign with a red border, containing a black silhouette of a fence or gate.",
    # ClassId 46
    "A triangular sign with a red border, containing a black silhouette of a steam train.",
    # ClassId 47
    "A rectangular sign with three diagonal red stripes.",
    # ClassId 48
    "A rectangular sign with two diagonal red stripes.",
    # ClassId 49
    "A rectangular sign with one diagonal red stripe.",
    # ClassId 50
    "A rectangular sign with diagonal red stripes.",
    # ClassId 51
    "A square blue sign with a large white letter 'P'.",
    # ClassId 52
    "A square blue sign with a white silhouette of a bus.",
    # ClassId 53
    "A square blue sign with a white first aid symbol (often a cross).",
    # ClassId 54
    "A square blue sign with a white silhouette of a telephone receiver.",
    # ClassId 55
    "A square blue sign with a white silhouette of a fuel pump.",
    # ClassId 56
    "A square blue sign with a white silhouette of a person sleeping in a bed.",
    # ClassId 57
    "A square blue sign with a white silhouette of a knife and fork.",
    # ClassId 58
    "A square blue sign with a white silhouette of a cup and saucer."
            ],
            "bangladesh_dataset":[
    # First batch
    "A speed limit sign indicating a maximum speed of 20.",
    "A 30 speed limit sign with a red circular border.",
    "A blue, circular road sign with the number 50, indicating a minimum speed limit.",
    "A road sign showing a speed limit of 60.",
    "A road sign indicating a 70 speed limit.",
    # Second batch
    "A road sign indicating a maximum speed limit of 80.",
    "A sign indicating the end of a 20 speed limit zone.",
    "A prohibitory road sign with a red circular border and a green symbol inside.",
    "A road sign indicating the end of a no-overtaking restriction.",
    "A \"No overtaking\" road sign.",
    "A road sign prohibiting overtaking by heavy goods vehicles.",
    "A triangular warning sign for a priority crossroads ahead.",
    "A yellow, diamond-shaped priority road sign.",
    "An inverted triangular \"Give Way\" or \"Yield\" sign.",
    "A red, octagonal stop sign.",
    "A \"No vehicles\" sign, which is a white circle with a red border.",
    "A road sign prohibiting entry for trucks.",
    "A circular red \"No Entry\" sign with a white horizontal bar.",
    "A triangular warning sign with an exclamation mark, indicating a general hazard.",
    # Third batch (visual descriptions)
    "A triangular sign with a red border, showing a black arrow that curves to the left.",
    "A triangular road sign with a red border, featuring a black arrow that curves to the right.",
    "A triangular sign with a red border, displaying a black S-shaped line.",
    "A triangular sign with a red border, showing the black silhouettes of two cars side-by-side.",
    "A triangular road sign with a red border, depicting a black car silhouette on a bumpy line.",
    "A triangular sign with a red border, featuring the silhouette of a person walking across a series of horizontal lines.",
    "A triangular road sign with a red border, showing the black silhouettes of two people, one larger and one smaller.",
    "A triangular sign with a red border, containing a symbol of a traffic light with red, yellow, and green circles.",
    "A triangular road sign with a red border, displaying the black silhouette of a person walking.",
    "A triangular sign with a red border, showing the black silhouette of a bicycle.",
    "A triangular road sign with a red border, featuring the black silhouette of a farm tractor.",
    "A triangular sign with a red border, containing a black snowflake symbol.",
    "A triangular road sign with a red border, showing the silhouette of a leaping deer.",
    "A white circular sign with a single black diagonal stripe across it.",
    "A blue circular sign with a white arrow that curves to the right.",
    "A blue circular sign containing a white arrow that points to the left.",
    "A blue circular sign with a white arrow pointing straight upwards.",
    "A blue circular sign displaying two white arrows, one pointing up and one pointing to the right.",
    "A blue circular sign showing two white arrows, one pointing straight ahead and the other to the left.",
    "A blue circular sign with a white arrow pointing diagonally downwards and to the right.",
    "A blue circular sign featuring a white arrow that points diagonally down and to the left.",
    "A blue circular sign with three white arrows forming a counter-clockwise circle.",
    "A white circular sign with a red border and a blue background, divided by a single red diagonal line.",
    "A dark, circular object, possibly a sign, with a thin vertical line through its center."
            ],
            "persian_dataset":[
    # 0-9
    "Blue circular sign with two white arrows pointing left and right",                         # 0_Compulsory Keep BothSide
    "Blue round sign with a thick white arrow bending left",                                    # 1_Compulsory Keep Left
    "Blue circular sign with a thick white arrow bending right",                                # 2_Compulsory Keep Right
    "Red-edged triangle with a black bicycle symbol inside",                                    # 3_Cycle crossing
    "Red triangular warning sign with an exclamation mark in the center",                       # 4_Danger
    "Inverted red-and-white triangle pointing downward",                                        # 5_Give Way
    "Red-bordered triangle depicting a black hump or bump graphic",                             # 6_Hump
    "Triangular warning sign with a black left-curving arrow",                                  # 7_Left Bend
    "Blue rectangle with a white arrow pointing to the left edge",                              # 8_Left Margin
    "Red circle with a black left-turn arrow struck through by a red line",                     # 9_Left Turn Prohibited

    # 10-19
    "Red-bordered white circle with '30' in bold black digits",                                 # 10_Maximum Speed 30
    "White circular sign edged in red, displaying '40' in large black font",                    # 11_Maximum Speed 40
    "Round white sign with a red border and the number '50' centered",                          # 12_Maximum Speed 50
    "Red-outlined white disc featuring '60' in thick black numerals",                           # 13_Maximum Speed 60
    "Circular white background, red border, and '70' prominently printed",                      # 14_Maximum Speed 70
    "White speed limit sign with a red rim and '80' in bold",                                   # 15_Maximum Speed 80
    "Red-rimmed white circle showcasing '90' in large black type",                              # 16_Maximum Speed 90
    "Red-banned symbol over a black motorcycle silhouette on white",                            # 17_MotorCycle Prohibited
    "Red circle with a solid white horizontal bar in the center",                               # 18_No Entry
    "Blue circle crossed by a red diagonal line over a black horn symbol",                      # 19_No Horn

    # 20-29
    "Red circle with a blue background and a red 'X' over a parked car",                        # 20_NO Stopping
    "Red-bordered white circle showing two parked cars crossed by a red line",                  # 21_NO Waiting
    "Rectangular blue sign with a large white arrow pointing right or left",                    # 22_One way Traffic
    "Blue square sign with a white 'P' in the center",                                          # 23_Park
    "Red circle with a blue interior and a red slash over a parked car",                        # 24_Park Forbidden
    "Blue square with a white silhouette of a walking figure",                                  # 25_Pedestrain
    "Triangular red-bordered sign with a black walking person symbol",                          # 26_Pedestrian crossing
    "Red triangular warning sign with a bold black right-curving arrow",                        # 27_Right Bend
    "Blue rectangular sign with a white arrow pointing to the right edge",                      # 28_Right Margin
    "Red circle with a black right-turn arrow crossed by a red line",                           # 29_Right Turn Prohibited

    # 30-39
    "Orange diamond-shaped sign with a black worker digging symbol",                            # 30_Road Work
    "Triangular red-bordered sign with three circular arrows",                                  # 31_Roundabouts
    "Yellow diamond-shaped sign with two black child figures walking",                           # 32_School
    "Pentagonal yellow sign with a black silhouette of children crossing",                      # 33_School Crossing
    "Red triangle with a black 'T' symbol and a right-side branch",                             # 34_Side Road Right
    "Red triangular sign with 'SLOW' in bold black letters",                                    # 35_Slow
    "White rectangle with a black camera icon and speed limit text",                            # 36_Speed Camera
    "Red octagon with 'STOP' in bold white uppercase letters",                                  # 37_STOP
    "Red circle with a black truck symbol crossed by a red line",                               # 38_Truck Prohibited
    "Rectangular blue sign with opposing white horizontal arrows",                              # 39_Two Way Traffic

    # 40-42
    "Blue square sign with a white U-shaped arrow",                                             # 40_U-Turn
    "White circular sign with a blue U-turn arrow symbol",                                      # 41_U-Turn Allowed
    "Red circle with a black U-turn arrow crossed by a red slash"                               # 42_U-turn Prohibited
            ],
        }


    def __init__(self, kaggle_datasets, hugging_face_datasets, local_datasets):
        """Initialize a DatasetCollection object
        
        Args:
            kaggle_datasets (List<KaggleDataset>): list of kaggle datasets.
            hugging_face_datasets (List<HuggingFaceDataset>): list of hugging face datasets.
            local_datasets (List<LocalDataset>): list of local datasets.
        """

        self.kaggle_datasets = kaggle_datasets
        self.hugging_face_datasets = hugging_face_datasets
        self.local_datasets = local_datasets


    def stream_all(self) -> Iterator:
        """Unified generator"""
        yield from self.stream_kaggle()
        yield from self.stream_hugging_face()
        yield from self.stream_local()


        

    def stream_kaggle(self) -> Iterator:
        """
        Generator that serves one kaggle dataset at a time
        and cleans memory after each rendering
        """
        for dataset in self.kaggle_datasets:  
            try:
                dataset_loaded = self._load_kaggle_dataset(dataset)
                yield dataset_loaded  
            finally:
                del dataset_loaded      # Explicitly remove the reference
                self._cleanup()  # Force the garbage collection


    def stream_hugging_face(self) -> Iterator:
        """
        Generator that serves one Hugging Face dataset at a time
        and cleans memory after each rendering
        """
        for dataset in self.hugging_face_datasets:  
            try:
                dataset_loaded = self._load_hugging_face_dataset(dataset)
                yield dataset_loaded  
            finally:
                del dataset_loaded      # Explicitly remove the reference
                self._cleanup()  # Force the garbage collection


    def stream_local(self) -> Iterator:
        """
        Generator that serves one local dataset at a time
        and cleans memory after each rendering
        """
        for dataset in self.local_datasets:  
            try:
                dataset_loaded = self._load_local_dataset(dataset)
                yield dataset_loaded  
            finally:
                del dataset_loaded      # Explicitly remove the reference
                self._cleanup()  # Force the garbage collection


    def _cleanup(self) -> None:
        """
        Pulisce la memoria in modo aggressivo
        """
        gc.collect()  # Forza la garbage collection

        



    def _load_split_images(self, split_folder):
        image_paths = []
        labels = []
        
        for label in os.listdir(split_folder):
            label_folder = os.path.join(split_folder, label)
            if os.path.isdir(label_folder):
                for file in os.listdir(label_folder):
                    if file.lower().endswith(('.ppm','.png','.jpg')):
                        image_paths.append(os.path.join(label_folder, file))
                        labels.append(label)
        
        return {"image": image_paths, "label": labels}



    def _load_kaggle_dataset(self, kaggle_dataset):
        """
        Return a loaded kaggle dataset given the path
        """
        print("Processing the Kaggle dataset: " + kaggle_dataset.dataset_name)
        dataset_desired_path = os.path.join(self.kaggle_dataset_path, kaggle_dataset.dataset_name)
        if not os.path.exists(dataset_desired_path):
            dataset_default_path = kagglehub.dataset_download(kaggle_dataset.dataset_path)
            shutil.move(dataset_default_path, dataset_desired_path)
        complete_path = os.path.join(dataset_desired_path, kaggle_dataset.local_subpath)
        dataset_loaded = self._load_split_images(complete_path)
        dataset_loaded = DatasetDict({
            "train": LoadDataset.from_dict(dataset_loaded).cast_column("image", Image())
        })
        dataset = self._filter_dataset(kaggle_dataset,dataset_loaded)
        kaggle_dataset.pandas_dataset = dataset
        del dataset
        del dataset_loaded
        return kaggle_dataset


    def _load_hugging_face_dataset(self, hugging_face_dataset):
        """
        Return a loaded Hugging Face dataset given the path
        """
        print("Processing the Hugging Face dataset: " + hugging_face_dataset.dataset_name)
        dataset_desired_path = os.path.join(self.hugging_face_dataset_local_path, hugging_face_dataset.dataset_name)
        dataset_loaded = load_dataset(hugging_face_dataset.dataset_path, cache_dir=dataset_desired_path)
        dataset = self._filter_dataset(hugging_face_dataset, dataset_loaded)
        hugging_face_dataset.pandas_dataset = dataset
        del dataset
        del dataset_loaded
        return hugging_face_dataset
    

    def _load_local_dataset(self, local_dataset):
        """
        Return a loaded local dataset given the path
        """
        print("Processing the local dataset: " + local_dataset.dataset_name)
        # Load Belgian Traffic Sign Dataset
        dataset_loaded_train = self._load_split_images(os.path.join(local_dataset.dataset_path,"train"))
        dataset_loaded_test = self._load_split_images(os.path.join(local_dataset.dataset_path,"test"))
        dataset_loaded = DatasetDict({
            "train": LoadDataset.from_dict(dataset_loaded_train).cast_column("image", Image()),
            "test": LoadDataset.from_dict(dataset_loaded_test).cast_column("image", Image())
        })
        dataset = self._filter_dataset(local_dataset, dataset_loaded)
        local_dataset.pandas_dataset = dataset
        del dataset
        del dataset_loaded
        return local_dataset
    
    def _filter_dataset(self, dataset, dataset_loaded):
        """
        Filter the given dataset removing the labels that have few instances and transform the dataset into a pandas dataframe.
        """
        if "test" in dataset_loaded.keys():
            dataset_loaded=concatenate_datasets([dataset_loaded["train"], dataset_loaded["test"]])
        else:
            dataset_loaded = dataset_loaded['train']

        df=dataset_loaded.to_pandas()

        # Rename the image and label column with the column "image" and "label"
        df.rename(columns={dataset.column_name_image:"image", dataset.column_name_label:"label"},inplace=True)
        
        # Get label frequencies (number of samples for each label)
        label_frequencies = [(i,df[df["label"]==i].shape[0]) for i in df["label"].unique()]

        # Remove labels with less than 20 samples
        df_filtered = df[df["label"].isin([i[0] for i in label_frequencies if i[1] >= 20])]
        
        # Convert str lables in int
        all_labels = df_filtered["label"].unique()
        mapping_labels = {label: i for i, label in enumerate(all_labels)}
        df_filtered["label"]  = [mapping_labels[i] for i in df_filtered["label"]]
        
        for ind, img in enumerate(tqdm(df_filtered["image"], desc="Processing images")):
            if img['bytes'] is None:
                img['bytes'] = open(img["path"],"rb").read()

        print("Number of filtered labels: ", df_filtered["label"].nunique())
        print("Number of removed labels: ",df[dataset.column_name_label].nunique()-df_filtered["label"].nunique())
        print("Number of samples in the dataset: ",df_filtered.shape[0])

        return df_filtered
    

In [5]:
import PIL.Image as ImagePIL
import io
import torch
from torch.utils.data import Dataset as DatasetTorch

class PandasDataset(DatasetTorch):
    def __init__(self, dataframe, transform=None):
        """
        Own class to represent the datasets used in the experimentation
        """
        self.dataframe = dataframe
        self.transform = transform
        self.num_classes = dataframe["label"].nunique()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if isinstance(idx, torch.Tensor):
            idx = idx.item() 

        image = ImagePIL.open(io.BytesIO(self.dataframe.iloc[idx]['image']["bytes"])).convert("RGB")
        label = self.dataframe.iloc[idx]['label']

        # Applica le trasformazioni
        if self.transform:
            image = self.transform(image)

        return image, label

<h2>Few-shot models

In [59]:
from ldc_train import *
import os
from sklearn.metrics import f1_score
import numpy as np

class FewShotDataset(PandasDataset):
    def __init__(self, dataframe, transform=None, shots=16):
        super().__init__(dataframe, transform)
        
        self.num_shots = shots

        self.loader = None
        self.classnames = dataframe["label"].unique()

    def set_loader(self, loader):
        self.loader = loader

class Config():
    def __init__(self, config):
        self.num_classes = config["num_classes"]
        self.seed = config["seed"]
        self.shots = config["shots"]
        self.backbone = config["backbone"]
        self.lr = config["lr"]
        self.batch_size = config["batch_size"]
        self.loss_lambda = config["loss_lambda"]  # last is regularization lambda
        self.fuse_type = config["fuse_type"]
        self.cache_dir = config["cache_dir"]

class FewShotTrasform(object):

    @staticmethod
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    @staticmethod
    def transform_train(size, scale=(0.8, 1.0)):
        funcs = [
            T.RandomResizedCrop(size=size, scale=scale, interpolation=InterpolationMode.BICUBIC),
            T.RandomHorizontalFlip(p=0.5), FewShotTrasform._convert_image_to_rgb, ToTensor(),
            Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]
        return Compose(funcs)

    @staticmethod
    def transform_test(size):
        funcs = [
            Resize(size, interpolation=InterpolationMode.BICUBIC),
            CenterCrop(size), FewShotTrasform._convert_image_to_rgb, ToTensor(),
            Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]
        return Compose(funcs)

    pass

class FewShotModel():
    transform = FewShotTrasform.transform_train(224)

    def __init__(self, shots, batch_size, classnames, template, dataset_name):
        self.config = Config({
            "num_classes":len(classnames),
            "seed":2024,
            "shots":shots,
            "backbone":"ViT-B/16",
            "lr":0.001,
            "batch_size":batch_size,
            "loss_lambda":[1.0, 1.0, 1.0, 1.0, 1.0],
            "fuse_type":2,
            "cache_dir":"./__few_shot_cache__"
        })

        self.clip_model, self.preprocess = clip.load(self.config.backbone, download_root=self.config.cache_dir,
                                                     num_classes=len(classnames), config=self.config)
        self.clip_model.eval()

        Tools.print("Getting cached textual weights W ...")
        self.text_feats = self.clip_classifier(
            os.path.join(self.config.cache_dir, f"{dataset_name}_{self.config.backbone}_textfeats.pt"),
            classnames, template, self.clip_model)
        
    def train_epoch(self, epoch):
        self.clip_model.adapter.train()
        self.clip_model.visual.adapter.train()

        train_acc, train_loss = AvgACC(), 0.0
        loss_list = [0, 0, 0, 0, 0]
        with tqdm(enumerate(self.train_loader), total=len(self.train_loader), desc=f"epoch {epoch}") as tqdm_train:
            for _, (images, labels) in tqdm_train:
                images, labels = images.cuda(), labels.cuda()
                clip_logits, mlp_logits, ada_logits, total_logits, weight = self.clip_model.my_forward(images, self.text_feats)
                
                loss, losses = self.get_loss(labels, clip_logits, mlp_logits, ada_logits, total_logits,
                                             lambda_value=self.config.loss_lambda)
                train_loss += loss.item()
                train_acc.step(mlp_logits, labels)

                for i, l in enumerate(losses):
                    loss_list[i] += l.item()
                tqdm_train.set_postfix(cur_loss=loss.item())

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                if self.scheduler:
                    self.scheduler.step()

            train_acc_result = train_acc.cal()
            train_loss = train_loss / len(self.train_loader)
            pass

        print(f"train acc={train_acc_result}, "
                    f"[l1_loss, ce_loss] => {[one / len(self.train_loader) for one in loss_list]}")
        return train_loss
        
    def train_model(self, dataset_loader, epochs=10, lr=0.001):
        self.optimizer = torch.optim.AdamW(self.clip_model.parameters(), lr=lr / 10, weight_decay=1e-4, eps=1e-4)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, epochs * len(dataset_loader))

        self.train_loader = dataset_loader

        for epoch in range(epochs):
            loss = self.train_epoch(epoch)
            print(f"Epoch: {epoch}, loss: {loss:.4f}, "
                        f"lr: {self.optimizer.state_dict()['param_groups'][0]['lr']:.8f}")
            pass
            
    
    def calculate_accuracy(self, dataset_loader):
        self.clip_model.eval()
        all_labels, all_logits, all_preds = [], [], []

        best_beta = None

        with torch.no_grad():
            with tqdm(enumerate(dataset_loader), total=len(dataset_loader), desc='Evaluate') as tqdm_eval:
                for _, (images, labels) in tqdm_eval:
                    clip_logits, mlp_logits, ada_logits, tot_logits, weight = self.clip_model.my_forward(images.cuda(),
                                                                                                 self.text_feats)
                    all_logits.append([clip_logits, mlp_logits, ada_logits, tot_logits])
                    all_labels.append(labels)

                    all_preds.append(torch.argmax(tot_logits, -1).cpu())
                    pass
                pass
            pass

        all_labels = torch.cat(all_labels, dim=0)
        all_preds = torch.cat(all_preds, dim=0).numpy()

        return f1_score(all_labels.cpu().numpy(),all_preds, average="macro")
    
    @staticmethod
    def cal_acc(logits, labels):
        pred = torch.argmax(logits, -1)
        acc_num = (pred == labels.cuda()).sum().item()
        return 1.0 * acc_num / len(labels)
    
    @staticmethod
    def fuse_logits(mlp_logits, clip_logits, beta=1.0):
        return beta * mlp_logits + (1 - beta) * clip_logits
    
    def search_hp(self, mlp_logits, clip_logits, all_labels, start=0, end=1, step=50):
        beta_list = [i * (end - start) / step + start for i in range(step + 1)]
        accs, best_beta, best_acc = [], start, 0.
        for beta in beta_list:
            logits = self.fuse_logits(mlp_logits, clip_logits, beta=beta)
            acc = self.cal_acc(logits, all_labels) * 100.
            accs.append((beta, acc))
            if acc > best_acc:
                best_acc = acc
                best_beta = beta
        return best_beta, accs[-1][-1], best_acc


    @staticmethod
    def get_loss(labels, clip_logits, mlp_logits, ada_logits, total_logits, lambda_value=[1.0, 1.0, 1.0, 1.0, 1.0]):
        ce_loss = F.cross_entropy(mlp_logits, labels) * lambda_value[0]
        ce_loss2 = F.cross_entropy(ada_logits, labels) * lambda_value[1]
        ce_loss3 = F.cross_entropy(total_logits, labels) * lambda_value[2]

        l1_loss1 = F.l1_loss(mlp_logits, clip_logits) * lambda_value[3]
        l1_loss2 = F.l1_loss(ada_logits, clip_logits) * lambda_value[4]

        loss = l1_loss1 + l1_loss2 + ce_loss + ce_loss2 + ce_loss3
        return loss, [l1_loss1, l1_loss2, ce_loss, ce_loss2, ce_loss3]
    
    @staticmethod
    def clip_classifier(feat_path, classnames, template, clip_model):
        if os.path.exists(feat_path):
            Tools.print(f"Loading texture features from {feat_path}")
            text_feats = torch.load(feat_path, map_location='cpu')
            return text_feats.cuda()

        with torch.no_grad():
            clip_weights = []
            for classname in classnames:
                classname = str(classname).replace('_', ' ')
                if isinstance(template, list):
                    texts = [t.format(classname) for t in template]
                elif isinstance(template, dict):
                    texts = template[classname]

                texts = clip.tokenize(texts).cuda()
                # prompt ensemble for ImageNet
                class_embeddings = clip_model.encode_text(texts)
                class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
                class_embedding = class_embeddings.mean(dim=0)
                class_embedding /= class_embedding.norm()
                clip_weights.append(class_embedding)
                pass

            clip_weights = torch.stack(clip_weights, dim=1).cuda()
            torch.save(clip_weights, Tools.new_dir(feat_path))

        return clip_weights


<h2>NON-Few-shot models</h2>

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTImageProcessor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#############################################
# Modified Neural Network Class
##############################################

class ModifiedNN(nn.Module):
    '''
    Parent class for all neural networks model that we will implement.
    It is a basic nn.Module with a training method which implements the training for neural networks.
    '''
    def __init__(self):
        super().__init__()

    def calculate_accuracy(self, data_loader):
        """
        Calculate accuracy of the model on a given dataset loader
        """
        self.eval()  # Set the model to evaluation mode
        y_pred = []
        y_true = []

        with torch.no_grad():  # Disable gradient computation for evaluation
            for batch in data_loader:
                X_batch, y_batch = batch

                # Forward pass
                outputs = self.forward(X_batch)

                # Get predictions (class with the highest score)
                _, predicted = torch.max(outputs, 1)

                y_pred.extend(predicted.numpy())
                y_true.extend(y_batch.numpy())

        # Calculate AP
        return f1_score(y_true,y_pred,average="macro")
    
    def training_loop(self, train_loader, optimizer, loss_function, epochs=10):
        """
        Train the model on the given loader using the given loss and optimizer
        """
        # Training loop
        for epoch in range(epochs):
            self.train()  # Set the model to training mode
            running_loss = 0.0

            for batch in train_loader:
                X_batch, y_batch = batch

                if type(y_batch) is not torch.Tensor:
                    y_batch = torch.Tensor(y_batch)

                # Forward pass
                predictions = self.forward(X_batch)
                loss = loss_function(predictions, y_batch)

                # Backward pass
                optimizer.zero_grad()  # Clear previous gradients
                loss.backward()        # Compute gradients
                optimizer.step()       # Update parameters

                running_loss += loss.item()

            # Calculate average training loss
            avg_train_loss = running_loss / len(train_loader)

            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Training Loss: {avg_train_loss:.4f}")
        
        return avg_train_loss

##############################################
# Utility functions
###############################################

def from_data_loader_to_numpy(data_loader):
    '''
    Method that concevts a torch data loader to two numpy arrays: X (samples) and y (labels).
    '''
    X_list = []
    y_list = []

    for batch in data_loader:
        X_batch, y_batch = batch

        X_list.append(X_batch.numpy())
        if isinstance(y_batch,tuple):
            y_list.append(np.array(list(y_batch)))
        else:
            y_list.append(y_batch.numpy())
    
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    
    return X, y

#############################################
# CNN
#############################################

class CNNModel(ModifiedNN):
    '''
    CNN for traffic sign classification. The input is a 32x32x3 image.
    '''

    # Transformation used to adapt an input to the desired size wanted by the model
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
    ])
    
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # padding=1 per "same"
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 5 * 5, num_classes)
        
    def forward(self, x):
        # Forward pass
        x = F.relu(self.conv1(x))  
        x = F.relu(self.conv2(x))  
        x = self.pool1(x)          
        x = self.dropout1(x)
        x = F.relu(self.conv3(x))  
        x = F.relu(self.conv4(x))  
        x = self.pool2(x)          
        x = self.dropout2(x)
        x = self.flatten(x)       
        x = self.fc(x)             
        return F.softmax(x, dim=1)
    
    def train_model(self, train_loader, epochs=10, lr=0.001):
        # set optimizer and loss
        optimizer = optim.Adam(self.parameters(), lr=lr)
        loss = nn.CrossEntropyLoss()
        return super().training_loop(train_loader, optimizer, loss, epochs)
        

#############################################
# MDCNN
#############################################

class MCDNN(ModifiedNN):
    '''
    MCDNN for traffic sign classification. The input is a 48x48x3 image.
    '''
    # Transformation used to adapt an input to the desired size wanted by the model
    transform = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
    ])

    def __init__(self, num_classes):
        super(MCDNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 150, kernel_size=7)
        self.bn1 = nn.BatchNorm2d(150)
        self.conv2 = nn.Conv2d(150, 200, kernel_size=4)
        self.bn2 = nn.BatchNorm2d(200)
        self.conv3 = nn.Conv2d(200, 300, kernel_size=4)
        self.bn3 = nn.BatchNorm2d(300)
        self.fc1 = nn.Linear(300 * 3 * 3, 350)
        self.bn4 = nn.BatchNorm1d(350)
        self.fc2 = nn.Linear(350, num_classes)
        self.pool = nn.MaxPool2d(2)
        self.conv_drop = nn.Dropout2d(p=0.2)
        self.fc_drop = nn.Dropout(p=0.5)

    def forward(self, x):
        # Forward pass
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.conv_drop(self.pool(x))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.conv_drop(self.pool(x))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv_drop(self.pool(x))
        x = x.view(-1, 300 * 3 * 3)
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.fc2(self.fc_drop(x))
        return F.log_softmax(x, dim=1)
    
    def train_model(self, train_loader, epochs=10, lr=0.001):
        # set optimizer and loss
        optimizer = optim.Adam(self.parameters(), lr=lr)
        loss = F.nll_loss
        return super().training_loop(train_loader, optimizer, loss, epochs)
    

#############################################
# KNN
#############################################

class KNNClassifier:
    # Transformation used to adapt an input to the desired size wanted by the model
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ])
    
    def __init__(self, num_classes, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None
        self.model = KNeighborsClassifier(n_neighbors=self.k)

    def forward(self, X):
        self.model.fit(self.X_train, self.y_train)

        # Predict on new data
        predictions = self.model.predict(X)

        return predictions
    
    def process_hog(self, X):
        # Compute the HOG features
        hog = cv2.HOGDescriptor(_winSize=(32,32), _blockSize=(8,8), 
                        _blockStride=(4,4), _cellSize=(4,4), 
                        _nbins=9)

        hog_x = []

        for x in X:
            gray_scale = cv2.cvtColor(x.reshape((32,32,3)), cv2.COLOR_RGB2GRAY)
            gray_scale = (gray_scale*255).astype(np.uint8).reshape((32, 32))
            x_hog = hog.compute(gray_scale)
            hog_x.append(x_hog)
        
        return hog_x

    def train_model(self, train_loader, epochs=10, lr=0.001):
        X, y = from_data_loader_to_numpy(train_loader)

        # Store training data for distance calculations
        self.X_train = self.process_hog(X)
        self.y_train = y
    
    def calculate_accuracy(self, data_loader):
        X, y = from_data_loader_to_numpy(data_loader)
        preds = self.forward(self.process_hog(X))

        return f1_score(y,preds,average="macro")


#############################################
# HOG SVM
#############################################

class HOG_SVM():
    """
    SVM model applied on the HOG features extracted from the images
    """
    # Transformation used to adapt an input to the desired size wanted by the model
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ])
    
    def __init__(self,num_classes):
        """
        Multiclass SVM whihc implements multiple classifier, one for each class, in a one-vs-rest scenario
        """
        self.model = SVC(kernel='linear', decision_function_shape="ovr", probability=True)
    
    def process_hog(self, X):
        # Compute the HOG features
        hog = cv2.HOGDescriptor(_winSize=(32,32), _blockSize=(8,8), 
                        _blockStride=(4,4), _cellSize=(4,4), 
                        _nbins=9)

        hog_x = []

        for x in X:
            gray_scale = cv2.cvtColor(x.reshape((32,32,3)), cv2.COLOR_RGB2GRAY)
            gray_scale = (gray_scale*255).astype(np.uint8).reshape((32, 32))
            x_hog = hog.compute(gray_scale)
            hog_x.append(x_hog)

        return hog_x
    
    def process_dataset(self, dataset_loader):
        X, y = from_data_loader_to_numpy(dataset_loader)
        
        return self.process_hog(X),y

    def forward(self, X):
        return self.model.predict(X)

    def train_model(self, train_loader, epochs=10, lr=0.001):
        hog_X, y = self.process_dataset(train_loader)

        self.model.fit(hog_X,y)

    def calculate_accuracy(self, data_loader):
        """
        Calculate accuracy of the model on a given dataset loader
        """
        hog_X, y = self.process_dataset(data_loader)

        pred = self.model.predict_proba(hog_X)
        pred_labels = [np.argmax(p) for p in pred]

        return f1_score(y,pred_labels, average="macro")

        

#############################################
# ViT
#############################################

#processor = ViTImageProcessor.from_pretrained("kelvinandreas/vit-traffic-sign-GTSRB")
#model = ViTForImageClassification.from_pretrained("kelvinandreas/vit-traffic-sign-GTSRB")


# Comparison using Paired t-test

In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from scipy.stats import t
from sklearn.base import clone
from sklearn.metrics import accuracy_score
from scipy.stats import ttest_rel

class TrafficSignRecognitionComparator:

    def __init__(self, few_shot_model, no_few_shot_model, dataset_iterator, no_few_shot_model_results=None, k_shot=16, n_folds=5):
        """
        Initializes the comparator for traffic sign recognition.

        Args:
        few_shot_model: Few-shot model to compare
        no_few_shot_model: nNo-few shot model to compare
        dataset_iterator: the iterator on the collection of datasets.
        no_few_shot_model_results: the AP (Average Precision) results obtained for the specified no-few shot model. (List of AP values)
        k_shot: Number of samples per class in few-shot learning
        n_folds: Number of folds for cross-validation
        """

        self.few_shot_model = few_shot_model
        self.no_few_shot_model = no_few_shot_model
        self.dataset_iterator = dataset_iterator
        self.no_few_shot_model_results = no_few_shot_model_results
        self.k_shot = k_shot
        self.n_folds = n_folds

        
    
        

    def _verify_number_of_samples(self, dataset):
        """Verify if the dataset has a number of samples that is sufficient to execute the train of the few-shot model.

        Returns
            result: true if all the datasets have a number of samples that is sufficient to execute the train of the few-shot model, false otherwise.
        
        """
        X = dataset.pandas_dataset['image']
        y = dataset.pandas_dataset['label']

        classes = np.unique(y)
        # We find the class that has the lowest number of samples in the dataset
        min_number_of_samples = float('inf')
        for cls in classes:
            # Take the indices corresponding to the samples of the current class
            cls_indices = np.where(y == cls)[0]
            number_of_samples_cls = len(cls_indices)
            if number_of_samples_cls < min_number_of_samples:
                min_number_of_samples = number_of_samples_cls
        # After finding the number of samples in the class with the fewest instances
        number_of_samples_each_fold = min_number_of_samples/self.n_folds
        number_of_samples_training_setCV = number_of_samples_each_fold * (self.n_folds-1)
        if number_of_samples_training_setCV  >= self.k_shot:
            return True
        else:
            return False


    def _prepare_few_shot_data(self, X_train, y_train):
        """Filter the given training set in order to obtain a subset of it that can be used to train the Few-shot model.

        Args:
            X_train: the features of the given training set.
            t_train: the labels of the given training set

        Returns
            X_train_few_shot: the featues of the new smaller training set.
            y_train_few_shot: the labels of the new smaller training set.
        
        """

       
        X_train_few_shot = []
        y_train_few_shot = []
        classes = np.unique(y_train)
        
        for cls in classes:
            # Take the indices corresponding to the samples of the current class
            cls_indices = np.where(y_train == cls)[0]

            if len(cls_indices) < self.k_shot:
                raise ValueError("The class" + cls + "does not have sufficient samples to execute the train of the few-shot model. Minimun number of samples required: " + self.k_shot + ", actual number of samples: " + len(cls_indices) + "!")


            # Choose self.k_shot indices from all the indices found 
            selected_random_indices = np.random.choice(cls_indices, size=self.k_shot, replace=False)

            X_train_few_shot.extend(X_train.iloc[selected_random_indices])
            y_train_few_shot.extend(y_train.iloc[selected_random_indices])
            
            
        return X_train_few_shot, y_train_few_shot
    
    
    def _evaluate_dataset(self, dataset):
        """Performs evaluation on a single dataset. It uses the stratified K-fold cross validation<.

        Args:
            dataset: dataset to evaluate.

        Returns:
            no_few_shot_model_mean_ap_results: the mean AP result obtained from the cross-validation process applied to the No few-shot model.
            few_shot_model_mean_ap_results: the mean AP result obtained from the cross-validation process applied to the few-shot model.
        """
        
        if(self._verify_number_of_samples(dataset))==False:
            raise ValueError("The dataset " + dataset-dataset.name + " does not have have a number of samples that is sufficient to execute the train of the few-shot model")

        X = dataset.pandas_dataset['image']
        y = dataset.pandas_dataset['label']


        # Lists to store the AP results from the individual iterations of the cross-validation process.
        no_few_shot_ap_resultsCV = [] # The AP results obtained from the No few-shot model
        few_shot_scores_ap_resultsCV = [] # The AP results obtained from the few-shot model


        # Create a StratifiedKFold object for cross-validation.
        # This ensures each fold has a balanced distribution of classes.
        # Data is shuffled before splitting to improve representativeness,
        # and a fixed random seed is used for reproducibility.
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=42)

        n_iter=0

        # For each iteration of the Cross validation
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("\nIteration: ",n_iter)

            # test dataset loader
            dataset_pandas_test = pd.DataFrame()
            dataset_pandas_test["image"]=X_test
            dataset_pandas_test["label"]=y_test
            
            if self.no_few_shot_model_results is None:
                # train dataset loader
                dataset_pandas_train = pd.DataFrame()
                dataset_pandas_train["image"]=X_train
                dataset_pandas_train["label"]=y_train
                train_dataset_torch = PandasDataset(dataset_pandas_train, transform=self.no_few_shot_model.transform)
                train_dataset_loader = torch.utils.data.DataLoader(train_dataset_torch, batch_size=64, shuffle=True)

                test_dataset_torch = PandasDataset(dataset_pandas_test, transform=self.no_few_shot_model.transform)
                test_dataset_loader = torch.utils.data.DataLoader(test_dataset_torch, batch_size=64, shuffle=True)

                # No few-shot model
                no_few_shot_model = self.no_few_shot_model(num_classes=train_dataset_torch.num_classes)
                # Train the model on the training set
                print("\n#######################################")
                print("Training non-few shot model...")
                print("Train dataset size: ",train_dataset_loader.__len__())
                print("Train dataset size: ",test_dataset_loader.__len__())
                no_few_shot_model.train_model(train_dataset_loader)
                print("Training complete!")
                # Make the predictionas on the test set
                no_few_shot_acc= no_few_shot_model.calculate_accuracy(test_dataset_loader)
                print("\nTest F1-macro: ",no_few_shot_acc)
                print("##########################################")
                no_few_shot_ap_resultsCV.append(1 - no_few_shot_acc) # Average Precision result calculated for a single iteration of the cross-validation process
            

            # Few-shot model

            # Obtain the filtered training set starting from the cross-validation training set
            few_shot_X_train, few_shot_y_train = self._prepare_few_shot_data(X_train, y_train)

            # train dataset loader
            dataset_pandas_train = pd.DataFrame()
            dataset_pandas_train["image"]=few_shot_X_train
            dataset_pandas_train["label"]=few_shot_y_train
            train_dataset_torch = FewShotDataset(dataset_pandas_train, transform=self.few_shot_model.transform, shots=self.k_shot)
            train_dataset_loader = torch.utils.data.DataLoader(train_dataset_torch, batch_size=64, shuffle=True)

            # test dataset loader
            test_dataset_torch = FewShotDataset(dataset_pandas_test, transform=self.few_shot_model.transform, shots=self.k_shot)
            test_dataset_loader = torch.utils.data.DataLoader(test_dataset_torch, batch_size=64, shuffle=True)

            few_shot_model = self.few_shot_model(shots=self.k_shot, batch_size=64, classnames=train_dataset_torch.classnames, template=DatasetCollection.templates[dataset.dataset_name], dataset_name=dataset.dataset_name)

            # Train the model on the (filtered) training set
            print("\n#######################################")
            print("Training few-shot model...")
            print("Train dataset size: ",train_dataset_loader.__len__())
            print("Train dataset size: ",test_dataset_loader.__len__())
            few_shot_model.train_model(train_dataset_loader)
            print("Training complete!")
            # Make the predictionas on the test set
            few_shot_acc= few_shot_model.calculate_accuracy(test_dataset_loader)
            print("\nTest F1-macro: ",few_shot_acc)
            print("#######################################")
            few_shot_scores_ap_resultsCV.append(1 - few_shot_acc)   # Average Precision result for a single iteration of the cross-validation process

            n_iter+=1

        few_shot_model_mean_ap_results = sum(few_shot_scores_ap_resultsCV)/self.n_folds
        if self.no_few_shot_model_results is None:
            no_few_shot_model_mean_ap_results =  sum(no_few_shot_ap_resultsCV)/self.n_folds
        else:
            no_few_shot_model_mean_ap_results = -1
            
        return no_few_shot_model_mean_ap_results, few_shot_model_mean_ap_results
    

    def run_comparison(self, confidence_level=0.05):
        """It runs the comparison on all datasets and returns the results.

        Args:
            confidence_level: confidence interval to use when perform the paired t-test.

        Returns:
            t-stat: the t-statistic value.
            t_critical
            no_few_shot_ap_results: List to store the AP results obtained from each dataset with the No few-shot model
            few_shot_ap_results: List to store the AP results obtained from each dataset with the Few-shot model
            result: boolen value where true means that there is a significant difference between the two models, false otherwise. 

        """

        # Lists to store the AP result obtained from each dataset
        no_few_shot_ap_results = [] # List to store the AP results obtained from each dataset with the No few-shot model
        few_shot_ap_results = [] # List to store the AP results obtained from each dataset with the Few-shot model
        if self.no_few_shot_model_results is not None:
            no_few_shot_ap_results = self.no_few_shot_model_results
        
        # For each dataset
        print("\nEVALUATING DATASETS USING CROSS-VALIDATION\n")
        print("\n#######################################")
        print(self.no_few_shot_model.__name__," VS ",self.few_shot_model.__name__)
        print("#######################################")
        
        for i, dataset in enumerate(self.dataset_iterator):
            no_few_shot_ap_result, few_shot_ap_result = self._evaluate_dataset(dataset)
            no_few_shot_ap_results.append(no_few_shot_ap_result)
            few_shot_ap_results.append(few_shot_ap_result)
        
        # Exeute the paired t-test
        number_of_datasets = len(self.dataset_iterator)
        mean_differences = abs(no_few_shot_ap_results - few_shot_ap_results)
        mean_differences = np.mean(mean_differences)
        std_differences = np.std(mean_differences, ddof=1)  # Std campionaria (corretta per n-1)

        t_stat = mean_differences / (std_differences / np.sqrt(number_of_datasets))
        dof = number_of_datasets - 1  # Degrees of freedom
        t_critical = t.ppf(1 - confidence_level/2, dof)  # Two-tailed: confidence_level/2

        result = abs(t_stat) > t_critical
        return t_stat, t_critical, no_few_shot_ap_results, few_shot_ap_results, result
    





# Main

In [58]:

def main():
   # Kaggle datasets
   indian_dataset = KaggleDataset("neelpratiksha/indian-traffic-sign-dataset", "indian_dataset", "image", "label", os.path.join("Indian-Traffic Sign-Dataset", "images"))
   persian_dataset = KaggleDataset("saraparsaseresht/persian-traffic-sign-dataset-ptsd", "persian_dataset", "image", "label", os.path.join("PTSD_Recognition/PTSD_Recognition", "train"))
   bangladesh_dataset = KaggleDataset("tithikhan/traffic", "bangladesh_dataset", "image", "label", "myData")
   kaggle_datasets_list = []
   kaggle_datasets_list.append(indian_dataset)
   kaggle_datasets_list.append(persian_dataset)
   kaggle_datasets_list.append(bangladesh_dataset)

   # Hugging face datasets
   chinese_dataset = HuggingFaceDataset("kuchidareo/chinese_trafficsign_dataset", "chinese_dataset", "image", "label")
   german_dataset = HuggingFaceDataset("bazyl/GTSRB", "german_dataset", "Path", "ClassId")
   russian_dataset = HuggingFaceDataset("eleldar/rtsd_cleaned", "russian_dataset", "image", "sign_class")
   hugging_face_datasets_list = []
   hugging_face_datasets_list.append(chinese_dataset)
   hugging_face_datasets_list.append(german_dataset)
   hugging_face_datasets_list.append(russian_dataset)

   # Local datasets
   belgian_dataset = LocalDataset("./dataset/datasets/belgium-traffic-sign", "belgian_dataset", "image", "label")
   local_datasets_list = []
   local_datasets_list.append(belgian_dataset)

   allDatasets = DatasetCollection(kaggle_datasets_list,hugging_face_datasets_list,local_datasets_list)

   # List of Non-Few-Shot models 
   non_few_shot_models = [CNNModel,MCDNN,KNNClassifier,HOG_SVM]
   few_shot_models = [FewShotModel]

   ##
   # MATTIA
   ##
   fs_CNN = TrafficSignRecognitionComparator(few_shot_models[0], non_few_shot_models[0], allDatasets.stream_all(), k_shot=16, n_folds=5)
   t_stat, t_critical, no_few_shot_ap_results, few_shot_ap_results, result = fs_CNN.run_comparison()
   
   fs_MDCNN = TrafficSignRecognitionComparator(few_shot_models[0], non_few_shot_models[1], allDatasets.stream_all(), k_shot=16, n_folds=5)
   t_stat, t_critical, no_few_shot_ap_results, few_shot_ap_results, result = fs_MDCNN.run_comparison()
   """
   ##
   # ALESSANDRO
   ##

   fs_KNN = TrafficSignRecognitionComparator(few_shot_models[0], non_few_shot_models[2], allDatasets.stream_all(), k_shot=16, n_folds=5)
   t_stat, t_critical, no_few_shot_ap_results, few_shot_ap_results, result = fs_KNN.run_comparison()
   """
   """
   fs_SVM = TrafficSignRecognitionComparator(few_shot_models[0], non_few_shot_models[3], allDatasets.stream_all(), k_shot=16, n_folds=5)
   t_stat, t_critical, no_few_shot_ap_results, few_shot_ap_results, result = fs_SVM.run_comparison()"""

   print(t_stat,t_critical,no_few_shot_ap_results,few_shot_ap_results,result)


if __name__ == "__main__":
   main()


EVALUATING DATASETS USING CROSS-VALIDATION


#######################################
CNNModel  VS  FewShotModel
#######################################
Processing the Kaggle dataset: indian_dataset


Processing images: 100%|██████████| 13971/13971 [00:03<00:00, 3555.43it/s]


Number of filtered labels:  58
Number of removed labels:  0
Number of samples in the dataset:  13971

Iteration:  0

#######################################
Training non-few shot model...
Train dataset size:  175
Train dataset size:  44
Epoch 1/10
  Training Loss: 4.0150
Epoch 2/10
  Training Loss: 4.0085
Epoch 3/10
  Training Loss: 4.0100
Epoch 4/10
  Training Loss: 4.0283
Epoch 5/10
  Training Loss: 4.0463
Epoch 6/10
  Training Loss: 4.0463
Epoch 7/10
  Training Loss: 4.0463
Epoch 8/10
  Training Loss: 4.0462
Epoch 9/10
  Training Loss: 4.0462
Epoch 10/10
  Training Loss: 4.0462
Training complete!

Test F1-macro:  0.0014421997091957962
##########################################
2025-06-08 11:37:02 Getting cached textual weights W ...
2025-06-08 11:37:02 Loading texture features from ./__few_shot_cache__\indian_dataset_ViT-B/16_textfeats.pt

#######################################
Training few-shot model...
Train dataset size:  15
Train dataset size:  44


epoch 0: 100%|██████████| 15/15 [00:59<00:00,  3.95s/it, cur_loss=13.3]


train acc=0.01939655172413793, [l1_loss, ce_loss] => [8.178938802083334, 0.3331217447916667, 4.22734375, 4.0796875, 4.054427083333334]
Epoch: 0, loss: 20.8734, lr: 0.00000000
Training complete!


...

epoch 0: 100%|██████████| 15/15 [01:20<00:00,  5.38s/it, cur_loss=12.3]


train acc=0.036637931034482756, [l1_loss, ce_loss] => [7.495572916666666, 0.30439453125, 3.9895833333333335, 4.000911458333333, 3.9174479166666667]
Epoch: 0, loss: 19.7115, lr: 0.00000000
Training complete!


Evaluate: 100%|██████████| 44/44 [03:21<00:00,  4.59s/it]



Test F1-macro:  0.002265666342613476
#######################################

Iteration:  2

#######################################
Training non-few shot model...
Train dataset size:  175
Train dataset size:  44
Epoch 1/10
  Training Loss: 4.0476


KeyboardInterrupt: 